<a href="https://colab.research.google.com/github/anaferreira744/DE-DP-ADF/blob/main/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [ ]:
%pip install faker

# Producer

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("/content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.option("checkpointLocation", "/content/lake/bronze/messages/checkpoint")
.foreachBatch(insert_messages)
.start()
)


query.awaitTermination(60)


ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-8-21be1766fbc9>", line 26, in insert_messages
    enrich.write.mode("append").format("parquet").save("/content/lake/bronze/messages/data")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-

False

In [ ]:
query.isActive

False

In [ ]:
query.stop()

In [ ]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/*")
df.show()
df.count()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-11 17:52:...|    0|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    2|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    4|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    1|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    3|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|   18|  RECEIVED|d4ef5cfa-c232-42a...|  OTHER|      2006|   1011|
|2024-12-11 17:53:...|   43|  RECEIVED|2ef6e612-a788-483...|  OTHER|      2013|   1040|
|2024-12-11 17:53:...|   36|  RECEIVED|d6ad29ec-cddb-4a2...|  OTHER|      2002|   1000|
|2024-12-11 17:53:...|   42|  RE

65

# Additional datasets

In [ ]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql import functions as F


# Definição do schema explícito para os dados
schema = StructType([
    StructField("event_type", StringType(), True),
    StructField("message_id", StringType(), True),
    StructField("channel", StringType(), True),
    StructField("country_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("timestamp", TimestampType(), True)
])



# Função para escrever os dados em dois caminhos diferentes (dados válidos e corrompidos)
def write_to_silver(batch_df, batch_id):

    batch_df=df.join(countries, on="country_id", how="left")

    # Dados corrompidos: event_type é null, vazio ou "NONE"
    corrupted_df = batch_df.filter(
        (F.col("event_type").isNull()) |
        (F.col("event_type") == "") |
        (F.col("event_type") == "NONE")
    )

    # Escreve os dados corrompidos
    corrupted_df.write.mode("append").format("parquet") \
        .partitionBy("date") \
        .save("/content/lake/silver/messages_corrupted/data")

    # Dados válidos: não corrompidos
    valid_df = batch_df.filter(~(
        (F.col("event_type").isNull()) |
        (F.col("event_type") == "") |
        (F.col("event_type") == "NONE")
    ))

    # Escreve os dados válidos
    valid_df.write.mode("append").format("parquet") \
        .partitionBy("date") \
        .save("/content/lake/silver/messages/data")

# Leitura do streaming de dados da camada Bronze
df_bronze = spark.readStream.format("parquet") \
    .schema(schema) \
    .load("content/lake/bronze/messages/data/*")

# Enriquecendo os dados com o nome do país e extraindo a data
df_enriched = enrich_data(df_bronze)
df_enriched = df_enriched.withColumn("date", F.to_date("timestamp"))

# Configuração do streaming para dividir os dados e escrever nos caminhos apropriados
query = (df_enriched.writeStream
    .outputMode("append")
    .trigger(processingTime="5 seconds")
    .option("checkpointLocation", "/content/lake/silver/messages/checkpoint")
    .foreachBatch(write_to_silver)
    .start()
)

# Executar o streaming por pelo menos 20 segundos
query.awaitTermination(20)



False

In [ ]:
df.show()
df.count()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-11 17:52:...|    0|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    2|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    4|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    1|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|    3|      NONE|2ef6e612-a788-483...|   PUSH|      2007|   1011|
|2024-12-11 17:52:...|   18|  RECEIVED|d4ef5cfa-c232-42a...|  OTHER|      2006|   1011|
|2024-12-11 17:53:...|   43|  RECEIVED|2ef6e612-a788-483...|  OTHER|      2013|   1040|
|2024-12-11 17:53:...|   36|  RECEIVED|d6ad29ec-cddb-4a2...|  OTHER|      2002|   1000|
|2024-12-11 17:53:...|   42|  RE

65

In [ ]:

def checking_silver_data():
    bronze_count = spark.read.format("parquet").load("/content/lake/bronze/messages/data/*").count()
    clean_count = spark.read.format("parquet").load("/content/lake/silver/messages/data/*").count()
    corrupted_count = spark.read.format("parquet").load("/content/lake/silver/messages_corrupted/data/*").count()

    assert bronze_count == clean_count + corrupted_count, "Dataframes doesn't matches. Did you run the code more than once? Be careful with append mode"
    print(f'Validation passed: Bronze [{bronze_count}] = Clean [{clean_count}] + Corrupted [{corrupted_count}]')

checking_silver_data()

AssertionError: Dataframes doesn't matches. Did you run the code more than once? Be careful with append mode

In [ ]:
# TODO

False

## Checking data

In [ ]:
# TODO

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [ ]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [ ]:
# report 1
# TODO

## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [ ]:
# report 2
# TODO

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

